In [2]:
import torch

In [13]:
#RESNET
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

img = read_image("C:/Users/nmarr/OneDrive/Documents/4thYear/490/ML/whitecar-snowforest.jpg")

# Step 1: Initialize model with the best available weights
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = [preprocess(img)]

# Step 4: Use the model and visualize the prediction
prediction = model(batch)[0]
labels = [weights.meta["categories"][i] for i in prediction["labels"]]
box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                          labels=labels,
                          colors="red",
                          width=4, font_size=30)
im = to_pil_image(box.detach())
im.show()

In [2]:
#RESNET VIDEO
from torchvision.io.video import read_video
from torchvision.models.video import r3d_18, R3D_18_Weights

vid, _, _ = read_video("C:/Users/nmarr/OneDrive/Documents/4thYear/490/ML/polarvideo.mp4", output_format="TCHW")
vid = vid[:32]  # optionally shorten duration

# Step 1: Initialize model with the best available weights
weights = R3D_18_Weights.DEFAULT
model = r3d_18(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(vid).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
label = prediction.argmax().item()
score = prediction[label].item()
category_name = weights.meta["categories"][label]
print(f"{category_name}: {100 * score}%")

sled dog racing: 58.94786715507507%


In [15]:
#YOLOv5
import cv2
import time
import torch
import yolov5

# Load the YOLOv5 model
model = yolov5.load("yolov5s.pt", device=torch.device("cpu"))

# Set up the video capture device (e.g. a webcam)
capture = cv2.VideoCapture(0)

# Run the object detection loop
while True:
    # Capture frame-by-frame
    ret, frame = capture.read()

    # Convert the frame to a PyTorch tensor
    frame = torch.from_numpy(frame).permute(2, 0, 1).float() / 255.0
    frame = frame.unsqueeze(0)

    # Run object detection on the frame
    detections = model.detect(frame)

    # Draw bounding boxes and labels on the frame
    for detection in detections:
        x1, y1, x2, y2, label, prob = detection
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label}: {prob:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Check for user input
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# When everything is done, release the capture
capture.release()
cv2.destroyAllWindows()



YOLOv5  2022-12-30 Python-3.9.13 torch-1.13.1 CPU

Fusing layers... 
YOLOv5s summary: 270 layers, 7235389 parameters, 0 gradients
Adding AutoShape... 


AttributeError: 'AutoShape' object has no attribute 'detect'